# cv2PL: Get Started

This jupyter notebook serves as a quick start guide of the cv2PL library.  
It demonstrates its capabilities as well as the limitations and what to pay attention to.  
This notebook was created based on [this](https://github.com/Xilinx/PYNQ-ComputerVision/tree/master/notebooks/computer_vision) template.

## Include cv2PL



In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import cv2PL as cv2

## The video subsystem with HDMI 
The library uses the video subsystem from the base PYNQ design.
If you want to learn all about its capabilities, use the notebooks 
https://github.com/Xilinx/PYNQ/tree/master/boards/Pynq-Z1/base/notebooks/video
provided by Xilinx as an introduction.  
You can access the video subsystem simply with *cv2.video*  
It contains the HDMI-in and HDMI-out interfaces.  
CAUTION: hdmi_in.start() will take some time and will fail if no incoming video signal is detected.


In [2]:
hdmi_in = cv2.video.hdmi_in
hdmi_out = cv2.video.hdmi_out

hdmi_in.configure(cv2.PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

print(hdmi_in.mode)

VideoMode: width=1280 height=720 bpp=8


## Run the original OpenCV Sobel 5x5 

In [44]:
import cv2 as openCV
import time

iterations = 10

start = time.time()
for i in range(iterations):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    openCV.Canny(inframe, 100, 100, outframe)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
end = time.time()
print("Frames per second using OpenCV:  " + str(iterations / (end - start)))

RuntimeError: DMA channel not started

## Run the cv2PL Sobel 5x5 in the Programmable Logic


In [16]:
import time
import cv2 as openCV
iterations = 10

while True:
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    if base.switches[0].read():
        hdmi_out.writeframe(inframe)
        inframe.freebuffer()
    else:
        start = time.time()
        if base.switches[1].read():
            cv2.Canny(inframe, 100, 100, outframe)
        else:
            openCV.Canny(inframe, 100, 100, outframe)
        end = time.time()
        cv2.putText(outframe, ("FPS: "+ str(1 / (end - start))), (100,100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3, cv2.LINE_AA)  
        inframe.freebuffer()
        hdmi_out.writeframe(outframe)



KeyboardInterrupt: 

## cv2PL and continous memory

The video subsystem returns images as [contiguous memory arrays](https://pynq.readthedocs.io/en/latest/pynq_libraries/xlnk.html).  
This allows the cv2PL library to stream the data directly through the hardware.  
If the image is a normal numpy ndarray and no destination is given, the library must execute two copy functions.  
This results in a perspicuous drop of the framerate but is still faster than the software version.

In [5]:
import numpy as np

image = np.ndarray(shape=(1080,1920),dtype=np.uint8) 

iterations = 10

start = time.time()
for i in range(iterations):
    sobel = cv2.Sobel(image,-1,1,0,ksize=5)
end = time.time()
print("Frames per second using cv2PL without CMA:  " + str(iterations / (end - start)))

Frames per second using cv2PL without CMA:  16.418144610118855


The solution to this problem is allocating contiguous memory arrays and use them as images.  
Don't forget to free them after use.

In [6]:
from pynq import Xlnk
xlnk = Xlnk()

image_buffer  = xlnk.cma_array(shape=(1080,1920), dtype=np.uint8)
return_buffer = xlnk.cma_array(shape=(1080,1920), dtype=np.uint8)

iterations = 10

start = time.time()
for i in range(iterations):
    cv2.Sobel(image_buffer,-1,1,0,ksize=5,dst=return_buffer)
end = time.time()
print("Frames per second using cv2PL with CMA:  " + str(iterations / (end - start)))

image_buffer.close()
return_buffer.close()

Frames per second using cv2PL with CMA:  65.67885150201688


## Clean up HDMI drivers

NOTE: This is needed to reset the HDMI drivers in a clean state. If this is not run, subsequent executions of this notebook may show visual artifacts on the HDMI out (usually a shifted output image)

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Clean up cv2PL
NOTE: This cleanup is needed because the library allocates contiguous memory and must free it. Otherwise, it may allocate all the available contiguous memory after including it a few times. The only solution is a reboot of the device, therefore do the cleanup ;) 

In [ ]:
cv2.close()